In [ ]:
pip install transformers torch transformers_stream_generator tiktoken einops accelerate redlines

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from redlines import Redlines
from IPython.display import display, Markdown, Latex
import torch
import warnings
warnings.filterwarnings("ignore")

### MODEL: llama2-13b-orca-8k-3319

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/llama2-13b-orca-8k-3319", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("OpenAssistant/llama2-13b-orca-8k-3319", torch_dtype=torch.float16, low_cpu_mem_usage=True)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/175 [00:00<?, ?B/s]

In [ ]:
# Sample customer email text
text = """
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter,
Emily, whose birthday was on March 15th. Despite your website promising delivery within three days, the product arrived only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me
in a difficult position with no gift for Emily’s special day.This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt
explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
"""

qa_questions = [
    "1. What was the reason for the customer's disappointment with WidgetCorp in the email?",
    "2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?"
]

prompt = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks.
2 - Identify the entities in the text delimited by triple backticks.
3 - Answer the following questions delimited by triple backticks based on the text.
4 - Translate the summary into French and back to English.

Separate your answers with line breaks.

Text:
```{text}```
Questions:
```{qa_questions}```
"""

### SUMMARIZATION TASK

In [ ]:
query = "Summarize the following text:"
prompt = query + "\n" + text
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)
print(tokenizer.decode(output[0], skip_special_tokens=True))



Summarize the following text: 
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday \ 
was on March 15th. Despite your website promising delivery within three days, the product arrived \ 
only yesterday, missing her birthday entirely. I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with no gift for Emily’s special day. This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt explanation and an assurance that such delays will not occur in the future. Sincerely, Jane Doe, 

My friend, Jane Doe, was unhappy because she ordered a "SuperWidget" from WidgetCorp for her daughter's birthday, but it arrived late. She tried calling an

### QUESTION ANSWERING TASK

In [ ]:
query = f"""Answer the two questions:```{qa_questions}```"""
prompt = query + "\n" + "\n" + text + "\n" + "\n"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=255)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Answer the two questions: 1. What was the reason for the customer's disappointment with WidgetCorp in the email? 2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery? 

Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday \ 
was on March 15th. Despite your website promising delivery within three days, the product arrived \ 
only yesterday, missing her birthday entirely. I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with no gift for Emily’s special day. This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt explanation and an assurance that such delays will not occur in the 

### NAME ENTITY RECOGNITION TASK

In [ ]:
query = "Identify the entities and their types (use standard NER terminology) in the following text:"
prompt = query + "\n" + text
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=255)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Identify the entities and their types (use standard NER terminology) in the following text: 
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday \ 
was on March 15th. Despite your website promising delivery within three days, the product arrived \ 
only yesterday, missing her birthday entirely. I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with no gift for Emily’s special day. This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt explanation and an assurance that such delays will not occur in the future. Sincerely, Jane Doe.

Entity types: 
1. Organization: WidgetCorp
2. Product: SuperWidget
3. Event: birthday
4. Date:

### TRANSLATION (English to French to English)

In [ ]:
query = "Translate the text two times: first from English to French, then translate the French text to English. Give both translations "
prompt = query + "\n" + "\n" + text + "\n" + "\n"
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=1200)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Translate the text two times: first from English to French, then translate the French text to English. Give both translations 

Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday \ 
was on March 15th. Despite your website promising delivery within three days, the product arrived \ 
only yesterday, missing her birthday entirely. I attempted to contact your customer service multiple times for an explanation and to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with no gift for Emily’s special day. This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt explanation and an assurance that such delays will not occur in the future. Sincerely, Jane Doe

Déar WidgetCorp,
Je vous écris pour exprimer ma profonde déc